In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Kim Wilde,Staying With My Baby,Written by ricky wilde \r\n \r\n(I'm gonna s...
1,Yazoo,In My Room,I stand alone and watch the clock \r\nI only ...
2,Backstreet Boys,Don't Disturb This Groove,"Ooh that sounds good, why don't you turn that ..."
3,Neil Sedaka,What A Surprise,"Do do do do do do do do, what a surprise \r\n..."
4,Clash,Revolution Rock,"Revolution rock, it is a brand new rock \r\nA..."
5,Zebra,Don't Walk Away,I don't know why you keep lying \r\nabout you...
6,The Weeknd,Earned It,[Hook] \r\nI'm so used to being used \r\nSo ...
7,The Killers,Smile Like You Mean It,"Save some face, you know you've only got one ..."
8,Frankie Goes To Hollywood,For Heaven's Sake,"Leave them behind, those clouds of trouble \r..."
9,Eric Clapton,Cryin',"[Chorus:] \r\nShe is cryin', she is cryin'. ..."


In [9]:
df['text'][0]

"Written by ricky wilde  \r\n  \r\n(I'm gonna stay... (I'm gonna stay...))  \r\n  \r\nThe pain inside  \r\nWhen you think you love another  \r\nBut can't decide which way to go  \r\nAnd you're searching for a reason to leave his side  \r\nIn time you'll see  \r\nIt has to be this way  \r\nAnd though it hurts to say  \r\nI'm gonna be staying with my baby  \r\n  \r\n(I'm gonna stay... (I'm gonna stay...))  \r\nI know he's good for me  \r\nI can see that he's feeling down  \r\n(I'm gonna stay... (I'm gonna stay...))  \r\nBut I know he cares for me  \r\nAnd in time he will come around  \r\nSo dry your eyes, before you turn away  \r\nYou must know that I'll always love you  \r\nBut now, I'm staying with my baby  \r\n  \r\nYou used to say  \r\nI should maybe take it further  \r\nAnd leave one day  \r\nMy family - but I'm responsible for them  \r\nThey depend on me  \r\nSo now you see, our love is history  \r\nI'm sad it has to end  \r\nBut I'm gonna be staying with my baby  \r\n  \r\n(I'm go

In [10]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [12]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [15]:
similarity[0]

array([1.        , 0.0057605 , 0.03824658, ..., 0.17791461, 0.12710417,
       0.0256915 ])

In [22]:
df[df['song'] == 'Staying With My Baby']

,artist,song,text
0,Kim Wilde,Staying With My Baby,written by ricki wild ( i 'm gon na stay ... (...


In [23]:
def recommendation(song_df):
    filtered_df = df[df['song'] == song_df]
    if filtered_df.empty:
        print(f"No songs found with the title '{song_df}'")
        return None  # or handle the case appropriately
    idx = filtered_df.index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    songs = []
    # Continue with your processing


In [24]:
recommendation('Staying With My Baby')

In [25]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))